<a href="https://colab.research.google.com/github/rnandodias/LangChain-Pandas-Dataframe-Agent/blob/main/LangChain_Pandas_Dataframe_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.7/520.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.9
    Uninstalling SQLAlchemy-2.0.9:
      Successfully uninstalled SQLAlchemy-2.0.9


In [2]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 5.7 MB/s eta 0:00:00


In [7]:
import os
os.environ["OPENAI_API_KEY"] = "COLOQUE SUA CHAVE AQUI"

In [3]:
from langchain.agents import create_pandas_dataframe_agent

In [4]:
from langchain.llms import OpenAI
import pandas as pd

In [5]:
df = pd.read_csv('aluguel.csv', sep = ';')

In [10]:
df.head()

,Tipo,Bairro,Quartos,Vagas,Suites,Area,Valor,Condominio,IPTU
0,Quitinete,Copacabana,1,0,0,40,1700.0,500.0,60.0
1,Casa,Jardim Botânico,2,0,1,100,7000.0,NaN,NaN
2,Conjunto Comercial/Sala,Barra da Tijuca,0,4,0,150,5200.0,4020.0,1111.0
3,Apartamento,Centro,1,0,0,15,800.0,390.0,20.0
4,Apartamento,Higienópolis,1,0,0,48,800.0,230.0,NaN


In [8]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

In [9]:
agent.run("Quantas linhas e colunas temos?")



> Entering new AgentExecutor chain...
Thought: I need to find out the shape of the dataframe
Action: python_repl_ast
Action Input: df.shape
Observation: (32960, 9)
Thought: I now know the final answer
Final Answer: O dataframe tem 32960 linhas e 9 colunas.

> Finished chain.


'O dataframe tem 32960 linhas e 9 colunas.'

In [11]:
agent.run("Quantos bairros temos no dataset?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of unique values in the Bairro column
Action: python_repl_ast
Action Input: len(df['Bairro'].unique())
Observation: 162
Thought: I now know the final answer
Final Answer: 162 bairros.

> Finished chain.


'162 bairros.'

In [13]:
agent.run("Qual o valor médio do aluguel dos Apartamentos na Barra da Tijuca?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include Apartamentos in Barra da Tijuca
Action: python_repl_ast
Action Input: df[(df['Tipo'] == 'Apartamento') & (df['Bairro'] == 'Barra da Tijuca')]['Valor'].mean()
Observation: 5586.852994011976
Thought: I now know the final answer
Final Answer: O valor médio do aluguel dos Apartamentos na Barra da Tijuca é 5586.85.

> Finished chain.


'O valor médio do aluguel dos Apartamentos na Barra da Tijuca é 5586.85.'

In [19]:
agent.run("""
    Construa uma tabela com os valores médios dos aluguéis
    dos Apartamentos em Copacabana segundo o total de quartos.
""")



> Entering new AgentExecutor chain...

Thought: I need to find the average rental values for apartments in Copacabana according to the number of bedrooms.
Action: python_repl_ast
Action Input: df[df['Tipo'] == 'Apartamento'][df['Bairro'] == 'Copacabana'].groupby('Quartos').mean()['Valor']
Observation: Quartos
0      940.000000
1     1998.556492
2     3509.844771
3     4775.150965
4     8649.464706
5    14814.666667
6     5250.000000
7    18000.000000
Name: Valor, dtype: float64
Thought:

<string>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<string>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.


 I now know the final answer
Final Answer: Os valores médios dos aluguéis dos Apartamentos em Copacabana segundo o total de quartos são: Quartos 0: 940.00, Quartos 1: 1998.56, Quartos 2: 3509.84, Quartos 3: 4775.15, Quartos 4: 8649.46, Quartos 5: 14814.67, Quartos 6: 5250.00 e Quartos 7: 18000.00.

> Finished chain.


'Os valores médios dos aluguéis dos Apartamentos em Copacabana segundo o total de quartos são: Quartos 0: 940.00, Quartos 1: 1998.56, Quartos 2: 3509.84, Quartos 3: 4775.15, Quartos 4: 8649.46, Quartos 5: 14814.67, Quartos 6: 5250.00 e Quartos 7: 18000.00.'